In [1]:
include("CayleyVerify.jl")

target_attack (generic function with 3 methods)

In [2]:
function dorefa_to_staircase(k::Int)
    n = 2^k - 1
    slopes = zeros(n+1)
    breakpoints = [-Inf]
    for i in 1:n
        push!(breakpoints, (2*i-1)/n - 1)
    end
    push!(breakpoints, Inf)
    
    constant_terms = [-1.0]
    for i in 1:n
        push!(constant_terms, -1.0 + 2*i/n)
    end
    return StaircaseFunction(breakpoints, slopes, constant_terms)
end

dorefa_to_staircase (generic function with 1 method)

In [17]:
using Pickle
using Suppressor
net_from_pickle = Pickle.load(open("./models/MNIST-DoReFa2:Dense256-Dense256.pkl"))
f = dorefa_to_staircase(2)
activation = [f, f]
neural_net = NeuralNetwork(net_from_pickle, activation)
print("net loaded")

net loaded

In [18]:
raw_imgs = Pickle.load(open("./imgs/MNIST:images-for-verification"))
imgs = []
for img in raw_imgs
    img = vcat([w' for w in img] ...)
    img = vcat(img'...)
    push!(imgs, img)
end
labels = Pickle.load(open("./imgs/MNIST:labels-for-verification", "r+"))
labels = Array{Int64}(labels.args[2][5])
labels = [l+1 for l in labels]
print("images loaded")

LoadError: [91mMethodError: no method matching iterate(::Nothing)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  iterate([91m::LibGit2.GitRevWalker[39m) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.5/LibGit2/src/walker.jl:29[39m
[91m[0m  iterate([91m::LibGit2.GitRevWalker[39m, [91m::Any[39m) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.5/LibGit2/src/walker.jl:29[39m
[91m[0m  iterate([91m::DataStructures.IntSet[39m) at /home/tuna/.julia/packages/DataStructures/DLSxi/src/int_set.jl:177[39m
[91m[0m  ...[39m

In [5]:
function predict(neural_net, img)
    num_layers = length(neural_net.weights)
    a = img'
    for i in 1:num_layers
        a = a * neural_net.weights[i] + neural_net.biases[i]'
        if i <= num_layers -1
            a = [eval(neural_net.activation[i], a[j]) for j in 1:length(a)]'
        end
    end
    output = a'
    return findmax(output)[2]
end

predict (generic function with 1 method)

In [6]:
using Printf
upper_bound = 150
lower_bound = 0
count = 1
for (img, label) in zip(imgs, labels)
    @printf("Verify %d-th image \n", count)
    vulnerable = false
    for target_label in 1:10
        if target_label != label
            @suppress begin
                opt_val, opt_sol_x, opt_sol_z = target_attack(neural_net, img, label, target_label, 0.016)
                if opt_val > 0
                    vulnerable = true
                    adv_img = [opt_sol_x[1, j] for j in 1:784]
                    pred = predict(neural_net, adv_img)
                    if pred != label
                        upper_bound -= 1
                        break
                    end
                end
            end
        end
    end
    if vulnerable == false
        lower_bound += 1
    end
    count += 1
end

Verify 1-th image 
Verify 2-th image 
Verify 3-th image 
Verify 4-th image 
Verify 5-th image 
Verify 6-th image 
Verify 7-th image 
Verify 8-th image 
Verify 9-th image 
Verify 10-th image 
Verify 11-th image 
Verify 12-th image 
Verify 13-th image 
Verify 14-th image 
Verify 15-th image 
Verify 16-th image 
Verify 17-th image 
Verify 18-th image 
Verify 19-th image 
Verify 20-th image 
Verify 21-th image 
Verify 22-th image 
Verify 23-th image 
Verify 24-th image 
Verify 25-th image 
Verify 26-th image 
Verify 27-th image 
Verify 28-th image 
Verify 29-th image 
Verify 30-th image 
Verify 31-th image 
Verify 32-th image 
Verify 33-th image 
Verify 34-th image 
Verify 35-th image 
Verify 36-th image 
Verify 37-th image 
Verify 38-th image 
Verify 39-th image 
Verify 40-th image 
Verify 41-th image 
Verify 42-th image 
Verify 43-th image 
Verify 44-th image 
Verify 45-th image 
Verify 46-th image 
Verify 47-th image 
Verify 48-th image 
Verify 49-th image 
Verify 50-th image 
Verify 51

In [9]:
lower_bound

130

In [10]:
upper_bound

146

In [11]:
upper_bouund

LoadError: UndefVarError: upper_bouund not defined